In [ ]:
!nvidia-smi
!pip install ultralytics roboflow -q
from ultralytics import YOLO
from roboflow import Roboflow
import os

Thu Feb 19 04:56:17 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   54C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:

rf = Roboflow(api_key="yD5IXmwv14ir3y3Abqfg")

project = rf.workspace("baka-1ravj").project("road-damage-det")
dataset = project.version(1).download("yolov8")

print(f"Dataset downloaded to: {dataset.location}")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to road-damage-dét-1 in yolov8:: 100%|██████████| 7836/7836 [00:01<00:00, 6959.73it/s]


Dataset downloaded to: /content/road-damage-dét-1


In [ ]:

model = YOLO('yolov8n.pt')

# Train the model
results = model.train(
    data=f'{dataset.location}/data.yaml',  # Path to your dataset config
    epochs=50,                              # Number of training epochs (start with 50)
    imgsz=640,                              # Training image size
    batch=16,                               # Batch size (adjust based on GPU memory)
    name='road_obstacles_yolov8n',          # Name for this training run
    patience=10,                            # Early stopping patience
    save=True,                              # Save checkpoints
    device=0,                               # Use GPU 0
    workers=2,                              # Number of dataloader workers
    project='runs/detect'                   # Where to save results
)

print("✅ Training complete!")
print(f"Best model saved at: runs/detect/road_obstacles_yolov8n/weights/best.pt")

Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.9.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/road-damage-dét-1/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=road_obstacles_yolov8n, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patie

In [ ]:
# Load the best trained model
model = YOLO('/content/runs/detect/runs/detect/road_obstacles_yolov8n/weights/best.pt')

# Validate on test set
metrics = model.val()

# Print metrics
print(f"\n📊 Validation Metrics:")
print(f"mAP50: {metrics.box.map50:.3f}")
print(f"mAP50-95: {metrics.box.map:.3f}")
print(f"Precision: {metrics.box.mp:.3f}")
print(f"Recall: {metrics.box.mr:.3f}")


Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.9.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
Model summary (fused): 73 layers, 3,007,013 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1600.6±790.6 MB/s, size: 61.4 KB)
val: Scanning /content/road-damage-dét-1/valid/labels.cache... 783 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 783/783 273.7Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 49/49 3.3it/s 14.8s
                   all        783       4264      0.728      0.705      0.738      0.504
    alligator cracking        379        592      0.783      0.801       0.85       0.69
         edge cracking        159        232      0.704      0.591      0.615       0.31
 longitudinal cracking        346        689      0.644       0.45      0.546      0.236
              patching        117        151      0.674      0.735      0.728      0.597
               pothole        574     

In [ ]:
model = YOLO('/content/runs/detect/runs/detect/road_obstacles_yolov8n/weights/best.pt')
print("🔄 Exporting to TFLite with INT8 quantization...")


model.export(
    format='tflite',
    imgsz=416,      # Smaller input = faster inference
    int8=True,      # INT8 quantization - critical!
    data=f'{dataset.location}/data.yaml'  # Needed for calibration
)

print("✅ Model exported: best_saved_model/best_int8.tflite")

# Check file size (should be ~3-4 MB for INT8)
import os
model_path = '/content/runs/detect/runs/detect/road_obstacles_yolov8n/weights/best_saved_model'
size_mb = os.path.getsize(model_path) / (1024**2)
print(f"📦 Model size: {size_mb:.2f} MB")

🔄 Exporting to TFLite with INT8 quantization...
Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.9.0+cu128 CPU (Intel Xeon CPU @ 2.00GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/
Model summary (fused): 73 layers, 3,007,013 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from '/content/runs/detect/runs/detect/road_obstacles_yolov8n/weights/best.pt' with input shape (1, 3, 416, 416) BCHW and output shape(s) (1, 11, 3549) (6.0 MB)
requirements: Ultralytics requirements ['sng4onnx>=1.0.1', 'onnx_graphsurgeon>=0.3.26', 'ai-edge-litert>=1.2.0', 'onnx>=1.12.0,<2.0.0', 'onnx2tf>=1.26.3,<1.29.0', 'onnxslim>=0.1.71', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
Using Python 3.12.12 environment at: /usr
Resolved 18 packages in 7.40s
Prepared 9 packages in 4.79s
Installed 9 packages in 259ms
 + ai-edge-litert==2.1.2
 + backports-strenum==1.3.1
 + colorama==0.4.6
 + onnx==1.20.1
 + onnx-

Exporting to ONNX opset version 22 is not supported. by 'torch.onnx.export()'. The highest opset version supported is 20. To use a newer opset version, consider 'torch.onnx.export(..., dynamo=True)'. 


ONNX: slimming with onnxslim 0.1.85...
ONNX: export success ✅ 2.0s, saved as '/content/runs/detect/runs/detect/road_obstacles_yolov8n/weights/best.onnx' (11.7 MB)
Unzipping calibration_image_sample_data_20x128x128x3_float32.npy.zip to /content/calibration_image_sample_data_20x128x128x3_float32.npy...: 100% ━━━━━━━━━━━━ 1/1 33.7files/s 0.0s
TensorFlow SavedModel: starting TFLite export with onnx2tf 1.28.8...
Saved artifact at '/content/runs/detect/runs/detect/road_obstacles_yolov8n/weights/best_saved_model'. The following endpoints are available:

* Endpoint 'serving_default'
  inputs_0 (POSITIONAL_ONLY): TensorSpec(shape=(1, 416, 416, 3), dtype=tf.float32, name='images')
Output Type:
  TensorSpec(shape=(1, 11, 3549), dtype=tf.float32, name=None)
Captures:
  139799572271312: TensorSpec(shape=(4, 2), dtype=tf.int32, name=None)
  139799572269776: TensorSpec(shape=(3, 3, 3, 16), dtype=tf.float32, name=None)
  139799572270544: TensorSpec(shape=(16,), dtype=tf.float32, name=None)
  139799572

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Test inference speed on the quantized model
import cv2
import numpy as np
import time
import os

# Check what files were created
print("📁 Checking export folder structure...")
!ls -la /content/runs/detect/runs/detect/road_obstacles_yolov8n/weights/best_saved_model/

# Correct path to your TFLite model
model_path = '/content/runs/detect/runs/detect/road_obstacles_yolov8n/weights/best_saved_model/best_int8.tflite'

# Check file size (should be ~3-4 MB for INT8)
if os.path.exists(model_path):
    size_mb = os.path.getsize(model_path) / (1024**2)
    print(f"✅ Model found!")
    print(f"📦 Model size: {size_mb:.2f} MB")

    if size_mb < 5:
        print("✅ INT8 quantization successful! (Model is small)")
    else:
        print("⚠️ Model might not be quantized properly (too large)")
else:
    print("❌ Model not found at expected path")
    print("Let's find it:")
    !find /content/runs/detect -name "*.tflite"

print("\n🧪 Model ready for Raspberry Pi testing!")
print("\nNext steps:")
print("1. Download the .tflite file (see next step)")
print("2. Transfer to Raspberry Pi")
print("3. Test with real-time inference")

📁 Checking export folder structure...
total 39408
drwxr-xr-x 4 root root     4096 Feb 13 09:57 .
drwxr-xr-x 3 root root     4096 Feb 13 09:55 ..
drwxr-xr-x 2 root root     4096 Feb 13 09:55 assets
-rw-r--r-- 1 root root  6126828 Feb 13 09:57 best_float16.tflite
-rw-r--r-- 1 root root 12167660 Feb 13 09:57 best_float32.tflite
-rw-r--r-- 1 root root  3242784 Feb 13 09:57 best_full_integer_quant.tflite
-rw-r--r-- 1 root root  3234176 Feb 13 09:57 best_int8.tflite
-rw-r--r-- 1 root root  3243048 Feb 13 09:57 best_integer_quant.tflite
-rw-r--r-- 1 root root       77 Feb 13 09:55 fingerprint.pb
-rw-r--r-- 1 root root      471 Feb 13 09:57 metadata.yaml
-rw-r--r-- 1 root root 12305667 Feb 13 09:55 saved_model.pb
drwxr-xr-x 2 root root     4096 Feb 13 09:55 variables
✅ Model found!
📦 Model size: 3.08 MB
✅ INT8 quantization successful! (Model is small)

🧪 Model ready for Raspberry Pi testing!

Next steps:
1. Download the .tflite file (see next step)
2. Transfer to Raspberry Pi
3. Test with real

In [ ]:

from google.colab import files

# Download the INT8 TFLite model directly
tflite_path = '/content/runs/detect/runs/detect/road_obstacles_yolov8n/weights/best_saved_model/best_int8.tflite'

print("📥 Downloading TFLite model...")
files.download(tflite_path)

print("✅ Download complete!")
print("File name: best_int8.tflite")
print("Transfer this to your Raspberry Pi!")

📥 Downloading TFLite model...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Download complete!
File name: best_int8.tflite
Transfer this to your Raspberry Pi!
